In [1]:
import networkx as nx
import numpy as np

In [10]:
with open("../data/21.txt") as f:
    codes = [line.rstrip() for line in f.readlines()]

In [11]:
def get_idx(r,c,nc):
    return (r*nc)+c

def invert_idx(idx,nc,nr):
    return (idx//nr,idx%nc)

def take_step(i,step,data,invalid_space="#"):
    nr,nc = data.shape
    move = (i[0]+step[0],i[1]+step[1])
    if move[0]>=0 and move[1]>=0 and move[0]<nr and move[1]<nc and data[move]!=invalid_space:
        
        return move
    
directions = [
    (-1,0), 
    (1,0),
    (0,1),
    (0,-1)
]

cardinals = {"^": (-1, 0), ">": (0, 1), "<": (0, -1), "v": (1, 0)}
# cardinals = {k:np.array(v) for k,v in cardinals.items()}
reverse_cardinals = {v:k for k,v in cardinals.items()}

In [12]:
num_pad="789\n456\n123\ng0A".split("\n")
arrow_pad="g^A\n<v>".split("\n")

nnr = 4
nnc = 3
num_pad = np.array([i for row in num_pad for i in row.rstrip()]).reshape((nnr,nnc))
num_pad

anr = 2
anc = 3
arrow_pad = np.array([i for row in arrow_pad for i in row]).reshape((anr,anc))


In [13]:
def build_small_graph(data):
    nr,nc=data.shape
    g = nx.Graph()
    i = 0
    idx_lkup = {}
    for r in range(nr):
        for c in range(nc):
            g.add_node(data[r,c])
            idx_lkup[data[r,c]] = np.array((r,c))
            i+=1

    # add edges
    for r in range(nr):
        for c in range(nc):
            if data[r,c] != "g":
                steps = [take_step((r,c),d,data,"g") for d in directions ]
                for step in steps:
                    if step:
                        g.add_edge(data[r,c],data[*step])
    return g,idx_lkup


In [14]:
n_g, num_lkup = build_small_graph(num_pad)
a_g, arrow_lkup = build_small_graph(arrow_pad)

In [15]:
test_cases = {
    "029A": "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A",
    "980A": "<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A",
    "179A": "<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
    "456A": "<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A",
    "379A": "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
}

In [16]:
# get all paths and prefer the one with repeated directions
# TODO: get all paths f rom tyhe num lookup, then sorted path for arrow lookups
# if  that doesnt work, maybe try the hardcoded best paths for numpad solution

# for problem 20
# find all paths starting with # and go until .
# if . == E, path is the path, otherwise, the path is the end + that block length to the end

In [17]:
def get_all_combos(lst):
    if not lst:
        return [[]]
    result = []
    for i in lst[0]:
        for j in get_all_combos(lst[1:]):
            result.append([i]+j)
    return result

def get_all_paths_for_code(code,n_g):
    num_pushes = []
    seqs = list(nx.all_pairs_all_shortest_paths(n_g))
    for i,c in enumerate(code):
        if i==0:
            src = "A"
            tgt = c
        else:
            src = code[i-1]
            tgt=c
        routes = [seq[1][tgt] for seq in seqs if seq[0]==src]
        num_pushes.append(routes[0])

    num_pushes = get_all_combos(num_pushes)
    paths = []
    for num_push in num_pushes:
        buttons = ""
        for seq in num_push:
            for i,p in enumerate(seq[:-1]):
                buttons += reverse_cardinals[tuple(int(t) for t in (num_lkup[seq[i+1]]-num_lkup[p]))]
            buttons += "A"
        paths.append(buttons)
    return paths

# get_all_paths_for_code(codes[0],n_g)

In [19]:
def get_button_pushes(code,start,n_g,lkup,use_sort=False):
    # start = "A"
    # code = codes[0]
    buttons = ""
    for c in code:
        path = nx.shortest_path(n_g,start,c)
        subs = ""
        for i,p in enumerate(path[:-1]):
            subs += reverse_cardinals[tuple(int(t) for t in (lkup[path[i+1]]-lkup[p]))]
        if use_sort and subs!="":
            buttons += "".join(sorted(subs))
        elif subs != "":
            buttons += subs
        buttons += "A"
        start = path[-1]
    return buttons

output = 0
for code in codes:
# code = codes[3]
    # r1 = get_button_pushes(code,"A",n_g,num_lkup,False)
    r1s = get_all_paths_for_code(code,n_g)
    r2s = []
    r3s = []
    for r1 in r1s:
        r2s.append(get_button_pushes(r1,"A",a_g,arrow_lkup,True))
        r3s.append(get_button_pushes(r2s[-1],"A",a_g,arrow_lkup,True))
        
    shortest = min([len(r3) for r3 in r3s])
    output += shortest*int(code.replace("A",""))
    # print(code, shortest,int(code.replace("A","")),len(test_cases[code])==shortest)
print(output)

171596
